In [1]:
!pip install numpy Pillow Matplotlib pandas seaborn scikit-learn requests scikit-image mlxtend prefixspan scikit-fuzzy umap-learn openpyxl stanza torch torchvision tesseract pytesseract nltk wordcloud spacy tensorflow==2.19.0 tensorflow-datasets opencv-python ucimlrepo nbconvert transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 23.1 MB/s eta 0:00:00
  Created wheel for prefixspan: filename=prefixspan-0.5.2-py3-none-any.whl size=11215 sha256=3d770e297114b90a156385812c4aeb812df5cade7fb7f7ba6ac8482d0beee74f
  Stored in directory: /root/.cache/pip/wheels/72/36/65/f9e2bbe4998ab4cdf6e7a015f38bdbb6925301dcf2cce34a88
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=a5bc65eee375aa53295d076dd717809835549d2d3e58b3b0dba3190ebefc2aab
  Stored in directory: /root/.cache/pip/wheels/13/1f/8e/2d6c0e358fd6d01ca80ecd9185a374bcda35

In [2]:
# podemos cargar el modelo directamente
import os
import cv2
import numpy as np
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import tempfile
import torch

In [3]:
print("cargando los modelos de reconocimiento de emociones y estimaciones")

# traigo modelos preentrenado donde nosotros vamos a ajustar a nuestras necesidades
processor_emotion = AutoImageProcessor.from_pretrained(
    "dima806/facial_emotions_image_detection"
)
modelo_emotion = AutoModelForImageClassification.from_pretrained(
    "dima806/facial_emotions_image_detection"
)

processor_age = AutoImageProcessor.from_pretrained("Robys01/facial_age_estimator")
model_age = AutoModelForImageClassification.from_pretrained(
    "Robys01/facial_age_estimator"
)

emotion_labels = modelo_emotion.config.id2label

cargando los modelos de reconocimiento de emociones y estimaciones


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [4]:
emotion_labels = {
    "sad": "Triste",
    "disgust": "Disgustado",
    "angry": "Enojado",
    "neutral": "Neutral",
    "fear": "Miedo",
    "surprise": "Sorpresa",
    "happy": "Feliz",
}

age_labels = {
    0: "01",
    1: "02",
    2: "03",
    3: "04-05",
    4: "06-07",
    5: "08-09",
    6: "10-12",
    7: "13-24",
    8: "26-40",
    9: "26-30",
    10: "31-35",
    11: "31-35",
    12: "36-40",
    13: "41-45",
    14: "45-50",
    15: "51-55",
    16: "56-60",
    17: "61-70",
    18: "71-80",
    19: "81-90",
    20: "90+",
}

print("etiquetas de emociones ", emotion_labels)
print("etiquetas de edades ", age_labels)

etiquetas de emociones  {'sad': 'Triste', 'disgust': 'Disgustado', 'angry': 'Enojado', 'neutral': 'Neutral', 'fear': 'Miedo', 'surprise': 'Sorpresa', 'happy': 'Feliz'}
etiquetas de edades  {0: '01', 1: '02', 2: '03', 3: '04-05', 4: '06-07', 5: '08-09', 6: '10-12', 7: '13-24', 8: '26-40', 9: '26-30', 10: '31-35', 11: '31-35', 12: '36-40', 13: '41-45', 14: '45-50', 15: '51-55', 16: '56-60', 17: '61-70', 18: '71-80', 19: '81-90', 20: '90+'}


In [5]:
src_path = os.path.join(cv2.data.haarcascades, "haarcascade_frontalface_default.xml")
temp_dir = tempfile.gettempdir()
dest_path = os.path.join(temp_dir, "haarcascade_frontalface_default.xml")

if not os.path.exists(dest_path):
    with open(src_path, "rb") as f_src:
        with open(dest_path, "wb") as f_dest:
            f_dest.write(f_src.read())

# cargamos nuestro clasificador
face_cascade = cv2.CascadeClassifier(dest_path)
if face_cascade.empty():
    raise RuntimeError("No se pudo cargar el clasificador Hear Cascade")

# activamos el activador de camara
cap = cv2.VideoCapture(0)

In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # tenemos que convertir los frames a escalas de grises para que se pueda hacer la identificacion de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # x e y son las coord que se encuentra en la parte sup del rostro
    # w h (ancho y alto)
    for x, y, w, h in faces:
        face_img = frame[y : y + h, x : x + w]
        face_img_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))

        inputs_emotion = processor_emotion(images=face_img_pil, return_tensors="pt")

        with torch.no_grad():
            outputs_emotion = modelo_emotion(**inputs_emotion)
        logits_emotion = outputs_emotion.logits
        predicted_class_idx_emotion = logits_emotion.argmax(-1).item()
        emotion_text = emotion_labels[
            list(emotion_labels.keys())[predicted_class_idx_emotion]
        ]

        inputs_age = processor_age(images=face_img_pil, return_tensors="pt")

        with torch.no_grad():
            outputs_age = model_age(**inputs_age)

        logits_age = outputs_age.logits
        predicted_class_idx_age = logits_age.argmax(-1).item()
        age_text = age_labels[predicted_class_idx_age]

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Frame {emotion_text}",
            (x, y - 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2,
        )
        cv2.putText(
            frame,
            f"Edad {age_text}",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2,
        )

    cv2.imshow("Reconomiento facial con edad", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()